# Exercise: Troll Treasure

In [1]:
def direction(point_a, point_b):
    """
    Returns the direction from point_a to point_b, or None if they
    are not neighhbouring grid points.
    """
    if (point_b[0] == point_a[0] - 1) and (point_b[1] == point_a[1]):
        return "left"
    if (point_b[0] == point_a[0] + 1) and (point_b[1] == point_a[1]):
        return "right"
    if (point_b[0] == point_a[0]) and (point_b[1] == point_a[1] - 1):
        return "up"
    if (point_b[0] == point_a[0]) and (point_b[1] == point_a[1] + 1):
        return "down"
    if point_b == point_a:
        return "nowhere"
    return None

In [2]:
class Room:
    def __init__(self, point, links=None):
        self.point = tuple(point)  # grid point of this room
        self.links = links  # other rooms this rooms connects to
        self._validate_links()

    def __contains__(self, point):
        return point in [l.point for l in self.links]

    def _validate_links(self):
        """
        Verifies all linked rooms are at neighbouring grid points
        """
        if not self.links:
            return
        for link in self.links:
            if not direction(self.point, link.point):
                raise ValueError(
                    f"Invalid link: {link.point} is not connected to {self.point}"
                )

In [3]:
class Rooms:
    """
    Collection of rooms
    """
    def __init__(self, rooms):
        # rooms dictionary keyed by (x, y) coordinate (grid cell indices)
        self.rooms = {r.point: r for r in rooms}

    def __iter__(self):
        """
        Allows Rooms objects to be iterated over (see Module 7)
        """
        return iter(self.rooms.values())

    def __getitem__(self, point):
        """
        rooms[(x, y)] will retrieve the room at coordinate (x, y) (where
        rooms is an instance of the Rooms class)
        """
        return self.rooms[point]

    def __contains__(self, point):
        """
        (x, y) in rooms will return True if a room at coordinates (x, y)
        is in rooms (where rooms is an instance of the Rooms class)
        """
        return point in self.rooms

    @classmethod
    def from_dict(cls, room_dict):
        rooms = []
        for point, links in room_dict.items():
            rooms.append(
                Room(point, [Room(l) for l in links])
            )
        return cls(rooms)

    @classmethod
    def from_list(cls, room_list):
        rooms = []
        for r in room_list:
            rooms.append(
                Room(r["point"], [Room(l) for l in r["links"]])
            )
        return cls(rooms)

In [4]:
class Treasure:
    def __init__(self, point, symbol):
        self.point = tuple(point)  # (x, y) grid location of the treasure
        self.symbol = symbol  # single char symbol to show the treasure on dungeon maps
    
    @classmethod
    def from_dict(cls, treasure_dict):
        return cls(treasure_dict["point"], treasure_dict["symbol"])
        

In [5]:
class Agent:
    """
    Base functionality to create and load (but not move) an Agent
    """

    def __init__(self, point, name, symbol, verbose=True, allow_wait=True, **kwargs):
        self.point = tuple(point)  # (x, y) grid location of the agent
        self.name = name  # e.g. adventurer or troll
        self.symbol = symbol # single char symbol to show the agent on dungeon maps
        self.verbose = verbose  # print output on agent behaviour if True
        self.allow_wait = allow_wait  # allow the agent to move nowhere

    def move(self, rooms):
        raise NotImplementedError("Use an Agent base class")

    @classmethod
    def from_dict(cls, agent_dict):
        return cls(
            agent_dict["point"],
            agent_dict["name"],
            agent_dict["symbol"],
            allow_wait=agent_dict["allow_wait"],
        )

In [6]:
import random


class RandomAgent(Agent):
    """
    Agent that makes random moves
    """
    def move(self, rooms):
        if not rooms[self.point].links:
            # this room isn't linked to anything, can't move
            if self.verbose:
                print(f"{self.name} is trapped")
            return

        # pick a random room to move to
        options = rooms[self.point].links
        if self.allow_wait:
            options.append(self)
        new_room = random.choice(options)

        if self.verbose:
            move = direction(self.point, new_room.point)
            print(f"{self.name} moves {move}")
        self.point = new_room.point


In [7]:
class HumanAgent(Agent):
    """
    Agent that prompts the user where to move next
    """
    def move(self, rooms):
        if not rooms:
            if self.verbose:
                print(f"{self.name} is trapped")
            return
        # populate movement options depending on available rooms
        if self.allow_wait:
            options = ["wait"]
        else:
            options = []
        if (self.point[0] - 1, self.point[1]) in rooms:
            options.append("left")
        if (self.point[0] + 1, self.point[1]) in rooms:
            options.append("right")
        if (self.point[0], self.point[1] - 1) in rooms:
            options.append("up")
        if (self.point[0], self.point[1] + 1) in rooms:
            options.append("down")

        # prompt user for movement input
        choice = None
        while choice not in options:
            choice = input(f"Where will {self.name} move \n{options}? ")
        
        # move the agent
        if choice == "left":
            self.point = (self.point[0] - 1, self.point[1])
        elif choice == "right":
            self.point = (self.point[0] + 1, self.point[1])
        elif choice == "up":
            self.point = (self.point[0], self.point[1] - 1)
        elif choice == "down":
            self.point = (self.point[0], self.point[1] + 1)


In [8]:
dungeon = {
    "rooms": {
        (0, 0): [(0, 1), (1, 0)],
        (0, 1): [(0, 0), (1, 1)],
        (1, 1): [(1, 0), (0, 1)],
        (1, 0): [(0, 0), (1, 1), (2, 0)],
        (2, 0): [(1, 0), (2, 1), (3, 0)],
        (2, 1): [(2, 0), (3, 1)],
        (3, 1): [(2, 1)],
        (3, 0): [(2, 0)],
    },
    "treasure": (1, 0),
    "adventurer": (0, 0),
    "troll": (3, 0),
}

"""
dungeon = {
    "treasure": (1, 0),  # Room 1 contains treasure
    "adventurer": (0, 0),  # The adventurer starts in room 0
    "troll": (2, 0),  # The troll starts in room 2
    "rooms": {
        (0, 0):  [(1, 0)], # Room zero connects to room 1
        (1, 0): [(0, 0), (2, 0)],  # Room one connects to rooms 0 and 2
        (2, 0): [(1, 0), (3, 0)],  # Room 2 connects to room 1 and 3
        (3, 0): [(2, 0)],
    },  # Room 3 connects to room 2
}


dungeon = {
    "treasure": (1, 0),  # Room 1 contains treasure
    "adventurer": (0, 0),  # The adventurer starts in room 0
    "troll": (2, 0),  # The troll starts in room 2
    "rooms": {
        (0, 0):  [], # Room zero connects to room 1
        (1, 0): [(2, 0)],  # Room one connects to rooms 0 and 2
        (2, 0): [(1, 0), (3, 0)],  # Room 2 connects to room 1 and 3
        (3, 0): [(2, 0)],
    },  # Room 3 connects to room 2
}
"""

'\ndungeon = {\n    "treasure": (1, 0),  # Room 1 contains treasure\n    "adventurer": (0, 0),  # The adventurer starts in room 0\n    "troll": (2, 0),  # The troll starts in room 2\n    "rooms": {\n        (0, 0):  [(1, 0)], # Room zero connects to room 1\n        (1, 0): [(0, 0), (2, 0)],  # Room one connects to rooms 0 and 2\n        (2, 0): [(1, 0), (3, 0)],  # Room 2 connects to room 1 and 3\n        (3, 0): [(2, 0)],\n    },  # Room 3 connects to room 2\n}\n\n\ndungeon = {\n    "treasure": (1, 0),  # Room 1 contains treasure\n    "adventurer": (0, 0),  # The adventurer starts in room 0\n    "troll": (2, 0),  # The troll starts in room 2\n    "rooms": {\n        (0, 0):  [], # Room zero connects to room 1\n        (1, 0): [(2, 0)],  # Room one connects to rooms 0 and 2\n        (2, 0): [(1, 0), (3, 0)],  # Room 2 connects to room 1 and 3\n        (3, 0): [(2, 0)],\n    },  # Room 3 connects to room 2\n}\n'

In [9]:
import yaml


class Dungeon:
    """
    Dungeon with:
    - Connected set of rooms on a square grid
    - The location of some treasure
    - An adventurer agent with an initial position
    - A troll agent with an initial position
    """

    def __init__(self, rooms, treasure, adventurer, troll, verbose=True):
        self.rooms = rooms
        self.treasure = treasure
        self.adventurer = adventurer
        self.troll = troll
        self.verbose = True

        # the extent of the square grid
        self.xlim = (
            min(r.point[0] for r in self.rooms),
            max(r.point[0] for r in self.rooms),
        )
        self.ylim = (
            min(r.point[1] for r in self.rooms),
            max(r.point[1] for r in self.rooms),
        )

        self._validate()

    def _validate(self):
        if self.treasure.point not in self.rooms:
            raise ValueError(f"Treasure{self.treasure.point} is not in the dungeon")
        if self.adventurer.point not in self.rooms:
            raise ValueError(
                f"{self.adventure.name}{treasure.point} is not in the dungeon"
            )
        if self.troll.point not in self.rooms:
            raise ValueError(f"{self.troll.name}{treasure.point} is not in the dungeon")

    @classmethod
    def from_file(cls, path):
        with open(path) as f:
            spec = yaml.safe_load(f)

        rooms = Rooms.from_list(spec["rooms"])
        treasure = Treasure.from_dict(spec["treasure"])

        agent_keys = ["adventurer", "troll"]
        agents = {}
        for agent in agent_keys:
            if spec[agent]["type"] == "random":
                agent_class = RandomAgent
            elif spec[agent]["type"] == "human":
                agent_class = HumanAgent
            else:
                raise ValueError(f"Unknown agent type {spec[agent]['type']}")
            agents[agent] = agent_class(**spec[agent])

        return cls(rooms, treasure, agents["adventurer"], agents["troll"])

    def update(self):
        """
        Move the adventurer and the troll
        """
        self.adventurer.move(self.rooms)
        self.troll.move(self.rooms)
        if self.verbose:
            print()
            self.draw()

    def outcome(self):
        """
        Check whether the adventurer found the treasure or the troll
        found the adventurer
        """
        if self.adventurer.point == self.troll.point:
            return -1
        if self.adventurer.point == self.treasure.point:
            return 1
        return 0

    def set_verbose(self, verbose):
        """Set whether to print output"""
        self.verbose = verbose
        self.adventurer.verbose = verbose
        self.troll.verbose = verbose

    def draw(self):
        """Draw a map of the dungeon"""
        layout = ""

        for y in range(self.ylim[0], self.ylim[1] + 1):
            for x in range(self.xlim[0], self.xlim[1] + 1):
                # room and character symbols
                if (x, y) in self.rooms:
                    if self.troll.point == (x, y):
                        layout += self.troll.symbol
                    elif self.adventurer.point == (x, y):
                        layout += self.adventurer.symbol
                    elif self.treasure.point == (x, y):
                        layout += self.treasure.symbol
                    else:
                        layout += "o"
                else:
                    layout += " "

                # horizontal connections
                if ((x, y) in self.rooms) and (((x + 1), y) in self.rooms[(x, y)]):
                    layout += " - "
                else:
                    layout += "   "

            # vertical connections
            if y < self.ylim[1]:
                layout += "\n"
                for x in range(self.xlim[0], self.xlim[1] + 1):
                    if ((x, y) in self.rooms) and ((x, y + 1) in self.rooms[(x, y)]):
                        layout += "|"
                    else:
                        layout += " "
                    if x < self.xlim[1]:
                        layout += "   "
                layout += "\n"

        print(layout)

In [10]:
import copy
from art import tprint, aprint


class Game:
    def __init__(self, dungeon):
        self.dungeon = dungeon

    def preamble(self):
        tprint("Troll Treasure\n", font="small")
        print(
            f"""
The {self.dungeon.adventurer.name} is looking for treasure in a mysterious dungeon.
Will they succeed or be dinner for the {self.dungeon.troll.name} that lurks there?

The map of the dungeon is below:
o : an empty room
o - o : connected rooms
{self.dungeon.troll.symbol} : {self.dungeon.troll.name}
{self.dungeon.adventurer.symbol} : {self.dungeon.adventurer.name}
{self.dungeon.treasure.symbol} : the treasure
            """
        )

    def run(self, max_steps=1000, verbose=True, start_prompt=False):
        dungeon = copy.deepcopy(self.dungeon)
        dungeon.set_verbose(verbose)
        if verbose:
            self.preamble()
            dungeon.draw()
            if start_prompt:
                input("\nPress enter to continue...")
            else:
                print("\nLet the hunt begin!")

        for turn in range(max_steps):
            result = dungeon.outcome()
            if result != 0:
                if verbose:
                    if result == 1:
                        print(
                            f"\n{self.dungeon.adventurer.name} gets the treasure and returns a hero!"
                        )
                        tprint("WINNER", font="small")
                    elif result == -1:
                        print(f"\n{self.dungeon.troll.name} will eat tonight!")
                        tprint("GAME OVER", font="small")
                return result
            if verbose:
                print(f"\nTurn {turn + 1}")
            dungeon.update()
        # no outcome in max steps (e.g. no treasure and troll can't reach adventurer)
        if verbose:
            print(
                f"\nNo one saw {self.dungeon.adventurer.name} or {self.dungeon.troll.name} again."
            )
            tprint("STALEMATE", font="small")

        return result

    def probability(self, trials=10000, max_steps=1000, verbose=False):
        outcomes = {-1: 0, 0: 0, 1: 0}
        for _ in range(trials):
            result = self.run(max_steps=max_steps, verbose=False)
            outcomes[result] += 1
        for result in outcomes:
            outcomes[result] = outcomes[result] / trials
        return outcomes

In [11]:
%%writefile test_win100.yml
# 100% win rate
treasure:
    point: [1, 0]
    symbol: "*"

adventurer:
    type: "random"
    symbol: "a"
    name: "Adventurer"
    point: [0, 0]
    allow_wait: False

troll:
    type: "random"
    symbol: "T"
    name: "Troll"
    point: [3, 0]
    allow_wait: False

rooms:
    - point: [0, 0]
      links:
        - [1, 0]
    
    - point: [1, 0]
      links:
        - [0, 0]
        - [2, 0]

    - point: [2, 0]
      links:
        - [1, 0]
        - [3, 0]
        
    - point: [3, 0]
      links:
        - [2, 0]
    
outcome:
    -1: 0
    0: 0
    1: 1

Overwriting test_win100.yml


In [12]:
%%writefile test_win50.yml
# 50% win rate
treasure:
    point: [1, 0]
    symbol: "*"

adventurer:
    type: "random"
    symbol: "a"
    name: "Adventurer"
    point: [0, 0]
    allow_wait: False

troll:
    type: "random"
    symbol: "T"
    name: "Troll"
    point: [2, 0]
    allow_wait: False

rooms:
    - point: [0, 0]
      links:
        - [1, 0]
    
    - point: [1, 0]
      links:
        - [0, 0]
        - [2, 0]

    - point: [2, 0]
      links:
        - [1, 0]
        - [3, 0]
        
    - point: [3, 0]
      links:
        - [2, 0]

outcome:
    -1: 0.5
    0: 0
    1: 0.5

Overwriting test_win50.yml


In [13]:
%%writefile test_lose100.yml
# 100% lose rate (no treasure access)
treasure:
    point: [3, 0]
    symbol: "*"

adventurer:
    type: "random"
    symbol: "a"
    name: "Adventurer"
    point: [0, 0]
    allow_wait: False

troll:
    type: "random"
    symbol: "T"
    name: "Troll"
    point: [2, 0]
    allow_wait: False

rooms:
    - point: [0, 0]
      links:
        - [1, 0]
    
    - point: [1, 0]
      links:
        - [0, 0]
        - [2, 0]

    - point: [2, 0]
      links:
        - [1, 0]
        
    - point: [3, 0]
      links: []

outcome:
    -1: 1
    0: 0
    1: 0

Overwriting test_lose100.yml


In [14]:
%%writefile test_stalemate.yml
# 100% stalemate (adventurer disconnected)
treasure:
    point: [3, 0]
    symbol: "*"

adventurer:
    type: "random"
    symbol: "a"
    name: "Adventurer"
    point: [0, 0]
    allow_wait: True

troll:
    type: "random"
    symbol: "T"
    name: "Troll"
    point: [2, 0]
    allow_wait: True

rooms:
    - point: [0, 0]
      links:
        - [1, 0]
    
    - point: [1, 0]
      links:
        - [0, 0]

    - point: [2, 0]
      links:
        - [3, 0]
        
    - point: [3, 0]
      links:
        - [2, 0]
    
outcome:
    -1: 0
    0: 1
    1: 0

Overwriting test_stalemate.yml


In [15]:
%%writefile dungeon.yml
treasure:
    point: [2, 0]
    symbol: "*"

adventurer:
    type: "random"
    symbol: "a"
    name: "Adventurer"
    point: [1, 2]
    allow_wait: False

troll:
    type: "random"
    symbol: "T"
    name: "Troll"
    point: [4, 2]
    allow_wait: True

rooms:
    - point: [0, 2]
      links:
        - [1, 2]
    
    - point: [1, 2]
      links:
        - [0, 2]
        - [1, 1]
        - [1, 3]

    - point: [1, 1]
      links:
        - [1, 2]
        - [2, 1]
        
    - point: [1, 3]
      links:
        - [1, 2]
        - [2, 3]

    - point: [2, 1]
      links:
        - [1, 1]
        - [2, 0]
        - [3, 1]
    
    - point: [2, 3]
      links:
        - [1, 3]
        - [3, 3]
        - [2, 4]

    - point: [2, 0]
      links:
        - [2, 1]

    - point: [3, 1]
      links:
        - [2, 1]
        - [3, 2]

    - point: [3, 3]
      links:
        - [2, 3]
        - [3, 2]

    - point: [2, 4]
      links:
        - [2, 3]

    - point: [3, 2]
      links:
        - [3, 1]
        - [3, 3]
        - [4, 2]

    - point: [4, 2]
      links:
        - [3, 2]

Overwriting dungeon.yml


In [16]:
d = Dungeon.from_file("dungeon.yml")
g = Game(d)
g.run(max_steps=10)

 _____            _  _   _____                                     
|_   _| _ _  ___ | || | |_   _| _ _  ___  __ _  ___ _  _  _ _  ___ 
  | |  | '_|/ _ \| || |   | |  | '_|/ -_)/ _` |(_-<| || || '_|/ -_)
  |_|  |_|  \___/|_||_|   |_|  |_|  \___|\__,_|/__/ \_,_||_|  \___|
                                                                   


The Adventurer is looking for treasure in a mysterious dungeon.
Will they succeed or be dinner for the Troll that lurks there?

The map of the dungeon is below:
o : an empty room
o - o : connected rooms
T : Troll
a : Adventurer
* : the treasure
            
        *           
        |        
    o - o - o       
    |       |    
o - a       o - T   
    |       |    
    o - o - o       
        |        
        o           

Let the hunt begin!

Turn 1
Adventurer moves left
Troll moves nowhere

        *           
        |        
    o - o - o       
    |       |    
a - o       o - T   
    |       |    
    o - o - o       
        |    

0

In [17]:
d = Dungeon.from_file("dungeon.yml")
g = Game(d)
g.probability(max_steps=10)
# -1: troll wins, 0: stalemate, +1: adventurer wins

{-1: 0.2251, 0: 0.5861, 1: 0.1888}